In [1]:
from flask import Flask, request, jsonify, render_template
import random
import tensorflow
from tensorflow.keras.models import load_model
import numpy as np
import librosa
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import os

In [2]:
app = Flask(__name__, template_folder='template', static_folder='static')
model=load_model(r'EmotionEmergency.h5')

In [3]:
@app.route('/')
def home():
    return render_template('index.html')

In [4]:
@app.route('/predict_',methods=['GET','POST'])
def predict_():

    # get file from POST request and save it
    # audio_file = request.files["file"]
    # file_name = str(random.randint(0, 100000))
    # audio_file.save(file_name)
    audio_file = r"C:\Users\saxen\Downloads\output.wav"

    Features = pd.read_csv(r'D:\SER\E2\features_Emergency.csv')
    Y = Features['labels'].values
    encoder = OneHotEncoder()
    Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()
    
    # NOISE
    def noise(data):
        noise_amp = 0.035*np.random.uniform()*np.amax(data)
        data = data + noise_amp*np.random.normal(size=data.shape[0])
        return data
    # # STRETCH
    # def stretch(data, rate=0.8):
    #     return librosa.effects.time_stretch(data, rate)
    # # PITCH
    # def pitch(data, sampling_rate, pitch_factor=0.7):
    #     return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

    def feat_ext(data):
        mfcc = np.mean(librosa.feature.mfcc(y=data, sr=22050).T, axis=0)
        return mfcc

    def get_feat(path):
        data, sample_rate = librosa.load(path, duration=5, offset=0.6)
        # normal data
        res1 = feat_ext(data)
        result = np.array(res1)
        #data with noise
        noise_data = noise(data)
        res2 = feat_ext(noise_data)
        result = np.vstack((result, res2))
        return result

    try:
        feature = get_feat(audio_file)
        test =np.expand_dims(feature, axis=2)
        livepreds = model.predict(test)
        livepredictions = (encoder.inverse_transform((livepreds)))
    except ValueError:
        livepredictions = ['Blank Call']
    os.remove(audio_file)


    return render_template('result.html', prediction_text='{}'.format(livepredictions[0]))


In [5]:
# @app.route('/results',methods=['POST'])
# def results():
#     data = request.get_json(force=True)
#     prediction = model.predict(data.values())
    
#     return jsonify(prediction)

In [6]:
if __name__ == "__main__":
    app.run(debug=True,use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Aug/2022 17:29:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 17:29:34] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 17:29:34] "GET /static/images/logo.png HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 17:29:35] "GET /static/images/footer_logo.PNG HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 17:29:35] "GET /static/js/main.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 17:29:35] "GET /static/images/favicon.png HTTP/1.1" 200 -


1/1 [==============================] - 2s 2s/step


127.0.0.1 - - [25/Aug/2022 17:29:48] "GET /predict_ HTTP/1.1" 200 -
127.0.0.1 - - [25/Aug/2022 17:29:48] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 17:29:48] "GET /static/js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 17:29:48] "GET /static/images/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [25/Aug/2022 17:29:48] "GET /static/images/footer_logo.PNG HTTP/1.1" 304 -
